In [32]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline

import tensorflow as tf
import pandas as pd
import json
import gc


In [33]:
save_directory = '/content/Saved_model'

In [3]:
tokenizer_fine_tuned = DistilBertTokenizer.from_pretrained(save_directory)

model_fine_tuned = TFDistilBertForSequenceClassification.from_pretrained(save_directory)

Some layers from the model checkpoint at /content/Saved_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/Saved_model and are newly initialized: ['dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
intent_class_names= ['cancel_order', 'change_order', 'change_shipping_address',
                  'check_cancellation_fee', 'check_invoice', 'check_payment_methods',
                  'check_refund_policy', 'complaint', 'contact_customer_service',
                  'contact_human_agent', 'create_account', 'delete_account',
                  'delivery_options', 'delivery_period', 'edit_account',
                  'get_invoice', 'get_refund', 'newsletter_subscription',
                  'payment_issue', 'place_order', 'recover_password', 'registration_problems',
                  'review', 'set_up_shipping_address', 'switch_account', 'track_order', 'track_refund']

In [35]:
def classify_intent(text):
  test_text = text
  predict_input = tokenizer_fine_tuned.encode(
      test_text,
      truncation = True,
      padding = True,
      return_tensors = 'tf'
  )

  output = model_fine_tuned(predict_input)[0]

  prediction_value = tf.argmax(output, axis = 1).numpy()[0]

  predicted_class_name = intent_class_names[prediction_value]
  return predicted_class_name

In [6]:
classify_intent("I forget my password")

'recover_password'

In [36]:
import random

def generate_response(intent: str, user_query: str = None) -> str:
    """
    Generate an appropriate response based on the identified intent.

    Args:
    - intent (str): The classified intent of the user's query.
    - user_query (str, optional): The original query from the user to add more context.

    Returns:
    - str: The generated response tailored to the user's intent.
    """

    # Define response templates for each intent
    response_templates = {
        'cancel_order': [
            "I'm sorry to hear that you'd like to cancel your order. Could you please provide your order ID so we can proceed?",
            "To cancel your order, please provide the order number. I'll handle it right away."
        ],
        'change_order': [
            "To make changes to your order, please specify what changes you'd like to make along with your order ID.",
            "Could you provide your order ID and details on what you want to change? I'll update your order accordingly."
        ],
        'change_shipping_address': [
            "Please provide your order ID and the new shipping address, and I'll update it for you.",
            "I can help you change the shipping address. Could you provide the order ID and the new address details?"
        ],
        'check_cancellation_fee': [
            "To check if there's a cancellation fee, please provide your order ID.",
            "I can check the cancellation fee for you. Can you provide the order number?"
        ],
        'check_invoice': [
            "To check your invoice, please provide your order ID or account details.",
            "I can retrieve your invoice. Could you provide the necessary details, such as your order number?"
        ],
        'check_payment_methods': [
            "We accept several payment methods including credit/debit cards, PayPal, and bank transfers. Would you like more details?",
            "Our payment methods include credit cards, debit cards, PayPal, and more. Can I assist you with something specific?"
        ],
        'check_refund_policy': [
            "Our refund policy allows refunds within 30 days of purchase. Would you like more details on this?",
            "We offer refunds within 30 days. If you need more information, let me know."
        ],
        'complaint': [
            "I'm sorry to hear that you're not satisfied. Could you please provide more details about your issue?",
            "We apologize for any inconvenience. Can you tell me more about your complaint so I can assist you?"
        ],
        'contact_customer_service': [
            "You can reach our customer service via phone at 123-456-7890 or email at support@example.com.",
            "For customer service, please call 123-456-7890 or email support@example.com."
        ],
        'contact_human_agent': [
            "I will connect you with a human agent shortly. Please hold on.",
            "Connecting you to a human agent now. Please wait a moment."
        ],
        'create_account': [
            "To create an account, please provide your email address and desired password.",
            "I can help you create an account. Could you please provide your email and a preferred password?"
        ],
        'delete_account': [
            "To delete your account, please provide your account email and any other verification details.",
            "I'm sorry to see you go. Could you confirm your account details so we can proceed with the deletion?"
        ],
        'delivery_options': [
            "We offer several delivery options including standard, express, and overnight shipping. Which one would you like?",
            "Our delivery options range from standard to express. Would you like more details?"
        ],
        'delivery_period': [
            "The delivery period depends on your location and the chosen delivery option. Typically, it ranges from 3 to 7 business days.",
            "Delivery usually takes between 3 to 7 business days, depending on your location and selected shipping method."
        ],
        'edit_account': [
            "To edit your account details, please provide the changes you want to make along with your current account information.",
            "I can assist you with editing your account. Please specify what details you would like to update."
        ],
        'get_invoice': [
            "To get your invoice, please provide your order ID.",
            "I can fetch your invoice. Could you provide your order number or account details?"
        ],
        'get_refund': [
            "To process a refund, please provide your order ID and the reason for the refund.",
            "Refunds can be initiated with your order ID and some additional information. Could you provide those details?"
        ],
        'newsletter_subscription': [
            "To subscribe to our newsletter, please provide your email address.",
            "I can add you to our newsletter. Could you provide your email?"
        ],
        'payment_issue': [
            "I'm sorry to hear you're having payment issues. Could you provide more details or a specific error message?",
            "Let’s solve your payment issue. Could you please describe the problem or provide any error messages you received?"
        ],
        'place_order': [
            "To place an order, please provide the product details and your preferred payment method.",
            "I can help you place an order. Could you tell me what you'd like to purchase and your payment method?"
        ],
        'recover_password': [
            "To recover your password, please provide your registered email address.",
            "I can help with password recovery. Could you provide your account's email address?"
        ],
        'registration_problems': [
            "I'm sorry to hear you're having trouble registering. Could you describe the issue?",
            "Let’s resolve your registration issue. Can you provide more details?"
        ],
        'review': [
            "We'd love to hear your feedback! Could you provide your review or comments?",
            "Please share your feedback with us. We appreciate your input."
        ],
        'set_up_shipping_address': [
            "To set up a shipping address, please provide your full address details.",
            "I can set up your shipping address. Could you provide the complete address information?"
        ],
        'switch_account': [
            "To switch accounts, please log out of the current account and log in with the other account credentials.",
            "You can switch accounts by logging out and logging back in with a different account. Need further assistance?"
        ],
        'track_order': [
            "To track your order, please provide your order ID.",
            "I can track your order for you. Could you provide the order number?"
        ],
        'track_refund': [
            "To track your refund status, please provide your order ID and any related details.",
            "Refund tracking requires your order ID. Could you provide that information?"
        ],
    }

    # Get random response from the list of responses for the given intent
    if intent in response_templates:
        response = random.choice(response_templates[intent])
    else:
        response = "I'm not sure how to help with that. Could you please provide more details?"

    return response






In [37]:
generate_response('recover_password','I forget my password')

'To recover your password, please provide your registered email address.'

In [41]:
def classify_intent_generate_response(text):
  intent = classify_intent(text)
  response = generate_response(intent, text)
  print('Intent : ',{intent})
  print('Response : ',{response})

In [44]:
classify_intent_generate_response('I want to delete my account')

Intent :  {'delete_account'}
Response :  {"I'm sorry to see you go. Could you confirm your account details so we can proceed with the deletion?"}


In [26]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the pre-trained DistilGPT-2 model and tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Function to generate response formatted as required
def generate_response(intent, user_input):
    # Define a concise and directive prompt for the 'cancel_order' intent
    if intent == "cancel_order":
        prompt = f"The customer wants to cancel their order. Provide a polite response. Start with 'Response:'. User query: {user_input}"

    # Encode the input text
    inputs = tokenizer.encode(prompt, return_tensors="pt")

    # Generate the response with control on output length
    outputs = model.generate(
        inputs,
        max_length=60,  # Limit length to focus on intent response
        num_return_sequences=1,
        do_sample=True
    )

    # Decode the generated text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Ensure the response is in the correct format
    if "Response:" not in response:
        response = "Response: " + response

    return response.strip()




In [30]:
# Example usage
intent = "cancel_order"
user_input = "I would like to cancel my order."
response = generate_response(intent, user_input)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [31]:
print(response)

The customer wants to cancel their order. Provide a polite response. Start with 'Response:'. User query: I would like to cancel my order. If I don't then I will cancel it."


So when I asked myself if I wouldn't get a letter from Amazon informing me of
